In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")
#Pkg.add("Ipopt")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using Ipopt

In [3]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:200
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [4]:
scenarios[1]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,122.83,151.992,1
2,132.91,143.459,1
3,132.9,142.587,0
4,125.03,146.618,1
5,120.07,150.136,1
6,139.83,151.102,1
7,164.23,145.927,0
8,186.66,143.261,1
9,186.95,142.279,0


In [5]:
scenarios[2]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,2.01,22.3372,1
2,1.38,18.6099,1
3,0.09,15.9499,0
4,0.08,13.4696,1
5,0.05,9.2429,1
6,0.0,5.90827,1
7,-0.38,4.51948,0
8,0.0,2.73391,1
9,0.0,1.1039,0


In [6]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

Row,hour,p_DA
,Int64,Float64


In [19]:
W = 200
hours = 24

24

In [20]:
# Create a new model with GLPK solver
model = Model(Ipopt.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(scenarios[i][hour,"price"] * p_DA[hour] + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1) - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) - 1000*delta_up[i,hour]*delta_down[i,hour]) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] - delta_down[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:    28800
Number of nonzeros in inequality constraint Jacobian.:     9648
Number of nonzeros in Lagrangian Hessian.............:     4800

Total number of variables............................:    14424
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     9600
Total number of inequality constraints...............:     9648
        inequality constraints with only lower bounds:     9624
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       24

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.53e+02 5.11e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [21]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
#println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

Objective value: 132670.3274058188
p_DA: [31.444967678107517, 48.2400059343857, 51.14903222808832, 199.99989387799542, 43.07419358552298, 10.595290304665902, 0.00010810239080774947, 47.51838711993024, 43.7362903001936, 37.720967746077726, 22.81691931170582, 37.53774206881511, 22.14696767709735, 22.673644714632534, 18.579112925401578, 19.107161245953655, 19.809677468576364, 44.66854838044988, 0.00010807496424397173, 15.832241899079326, 53.61451681623334, 199.99989391261227, 53.84596774943079, 0.00010841900454442582]
delta_up: [120.54745163447491 95.21918760400366 91.43774195546241 -9.995860400806972e-9 107.06177409802739 140.5063225240453 145.92682733922356 95.74274188620107 98.54241931884083 104.4943547600534 118.76179030732825 100.77999984054186 115.18819358709807 114.13280687214358 114.41798380653583 110.42203221179024 102.02032247303866 70.45741928697254 108.50134346342483 87.70872583285862 44.56774122215861 -9.99799496130046e-9 30.80677414380203 78.06940768390146; -9.97626569942719